In [1]:
import pandas as pd
import os
import openai
from tqdm import tqdm
import time
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

df_texts = pd.read_csv('../data/touche-to-classify.csv', index=False)

openai.api_key = "API_KEY" # your OpenAI API key

In [7]:
df_texts.head()

,Topic,ID,Score,text,qid,topic_text,description,narrative,object1,object2
0,12,clueweb12-0002wb-18-34442___2,NO,Divided by length ostensibly to fit the amount...,12,Train or plane? Which is the better choice?,A frequent traveler (domestic and internationa...,A highly relevant document will contrast the e...,Train,plane
1,12,clueweb12-0004wb-69-30215___112,NO,"As he says ""some choice is better than no choi...",12,Train or plane? Which is the better choice?,A frequent traveler (domestic and internationa...,A highly relevant document will contrast the e...,Train,plane
2,12,clueweb12-0004wb-78-20304___1,SECOND,Europe -- By Plane or By Train? | IndependentT...,12,Train or plane? Which is the better choice?,A frequent traveler (domestic and internationa...,A highly relevant document will contrast the e...,Train,plane
3,12,clueweb12-0004wb-78-20304___11,NEUTRAL,The Bottom Line We learned quite a few things ...,12,Train or plane? Which is the better choice?,A frequent traveler (domestic and internationa...,A highly relevant document will contrast the e...,Train,plane
4,12,clueweb12-0008wb-62-05967___1,NO,"Most Amazing Truck, Train & Plane Songs home a...",12,Train or plane? Which is the better choice?,A frequent traveler (domestic and internationa...,A highly relevant document will contrast the e...,Train,plane


In [9]:
# Prompt GPT-3

predictions = list()

for idx, row in tqdm(df_texts.iterrows()):
    obj1 = row.object1
    obj2 = row.object2
    question = row.topic_text
    answer = row.text
    text = f'''
    You will be shown a text passage that compares two objects. 
    Decide if the passage provides arguments pro first object, pro second object, neutral, or no stance is given. 
    First, we start with examples and definitions. Please read them carefully.
    
    QUESTION: Apple vs Microsoft: which do you like better?
    ANSWER PASSAGE: I switched from PC to Mac about 2 years ago, after becoming familiar with Macs using my sister's computer. I will NEVER go back to PCs. I also like that Macs are simplified for basic things such as photos, music, internet and e-mail. Truthfully, the only programs I have issues with are Microsoft applications like Word and IE. I think Apple's superiority comes from the fact that Macs are inherently more stable systems.
    FIRST OBJECT: Apple;      SECOND OBJECT: Microsoft.
    Explanation: The answers provides a strong pro argument (opinion) for MAC (which is referred to Apple). Note, that the text passage may not use the same object names as the question, e.g., it can contain synonyms or abbreviation or just mention only one object.
    Stance: PRO FIRST OBJECT

    QUESTION: Is it better to dual-boot or run a VM?
    ANSWER PASSAGE: Dual boot is a waste of time. I describe it to people as "the 5-minute alt-tab". It's a pain to configure, and because you can't run both OSes at once, when you need the one you're not running, you have to kill off every app and reboot. I avoid dual boot like the plague. VM all the way. Or, just use a single OS that does what you want. Windows with Cygwin provides a lot of the Unixy stuff that most people need.
    FIRST OBJECT: to dual-boot;      SECOND OBJECT: run a VM.
    Explanation: The answers provides a strong opinion that a VM is better than a dual-boot. Note, that the text passage may not use the same object names as the question, e.g., it can contain synonyms or abbreviation or just mention only one object.
    Stance: PRO SECOND OBJECT

    QUESTION: Who would win in a battle, a squirrel or a bird?
    ANSWER PASSAGE: First of all, it depends on the bird size. The bird has an initial advantage of flying away. If the bird is big like a hawk, it would be a winner without any questions. But if it is small, it would fly away. And you know, the winner never runs away from the battlefield.
    FIRST OBJECT: squirrel;      SECOND OBJECT: bird.
    Explanation: The answers suggests that under some condition a bird would win, but without the condition a squirrel would. This means both could win a fight, and they are equal. 
    Stance: NEUTRAL

    QUESTION: Which to choose a pie or a tart?
    ANSWER PASSAGE: Generally speaking, a pie refers to a pastry covered with a lid, like a typical apple pie. A tart is open topped, like a quiche, or a French tartes aux pommes. However, there are exceptions to this: many pies will be open topped too. Usually this is a matter of depth: the deeper it is, the more likely it is to be called a pie rather than a tart. Regional variations also apply.
    FIRST OBJECT: pie;      SECOND OBJECT: tart.
    Explanation: The answer does not provide any pro or con arguments or opinions. The answer simply describes what a pie and a tart are. According to the definition of stance (see above), there is no stance in the passage.
    Stance: NO STANCE
    
    Also, select NO STANCE if the text passage does not contain arguments / opinions toward the objects (that is neither first nor second object nor their synonyms are in the text.)

    Now, I have a question comparing FIRST OBJECT: {obj1} and SECOND OBJECT: {obj2}:
    QUESTION: {question}
    Identify whether the following text is "PRO FIRST OBJECT", "PRO SECOND OBJECT", "NEUTRAL", or "NO STANCE". Please, answer only with "PRO FIRST OBJECT", "PRO SECOND OBJECT", "NEUTRAL", or "NO STANCE":
    ANSWER PASSAGE: {answer}
    Stance: 
    '''
    response = openai.Completion.create(
      model="text-davinci-003",
      prompt=text,
      temperature=0,
      max_tokens=64,
      top_p=1.0,
      frequency_penalty=0.0,
      presence_penalty=0.0
    )
    predictions.append(response["choices"][0]["text"].strip())
    time.sleep(3)
#    print(text)
#    print()

2107it [00:00, 14530.73it/s]


    You will be shown a text passage that compares two objects. 
    Decide if the passage provides arguments pro first object, pro second object, neutral, or no stance is given. 
    First, we start with examples and definitions. Please read them carefully.
    
    QUESTION: Apple vs Microsoft: which do you like better?
    ANSWER PASSAGE: I switched from PC to Mac about 2 years ago, after becoming familiar with Macs using my sister's computer. I will NEVER go back to PCs. I also like that Macs are simplified for basic things such as photos, music, internet and e-mail. Truthfully, the only programs I have issues with are Microsoft applications like Word and IE. I think Apple's superiority comes from the fact that Macs are inherently more stable systems.
    FIRST OBJECT: Apple;      SECOND OBJECT: Microsoft.
    Explanation: The answers provides a strong pro argument (opinion) for MAC (which is referred to Apple). Note, that the text passage may not use the same object names as the 

In [19]:
# Convert GPT-3 predictions into stance labels

print(len(predictions))
predictions_convert = list()
for pred in predictions:
    if 'first' in pred.lower():
        predictions_convert.append('FIRST')
    elif 'second' in pred.lower():
        predictions_convert.append('SECOND')
    elif 'neutral' in pred.lower():
        predictions_convert.append('NEUTRAL')
    elif 'no stance' in pred.lower():
        predictions_convert.append('NO')
print(len(predictions_convert))

23
23


In [34]:
# Convert ground truth stance labels into numerical values

def convert_labels_tonum(row):
    col = 'Score'
    if row[col] == 'NO':
        return 0
    elif row[col] == 'NEUTRAL':
        return 1
    elif row[col] == 'FIRST':
        return 2
    elif row[col] == 'SECOND':
        return 3
    
df_texts['Score_num'] = df_texts.apply(convert_labels_tonum, axis=1)

In [35]:
# Convert ground GPT-3 stance labels into numerical values

def convert_labels_tonum(row):
    col = 'gpt_pred_conv'
    if row[col] == 'NO':
        return 0
    elif row[col] == 'NEUTRAL':
        return 1
    elif row[col] == 'FIRST':
        return 2
    elif row[col] == 'SECOND':
        return 3
    
df_texts['gpt_pred_num'] = df_texts.apply(convert_labels_tonum, axis=1)

In [5]:
print(classification_report(y_true=df_texts.Score.tolist(), y_pred = df_texts.gpt_pred_conv.tolist()))

              precision    recall  f1-score   support

       FIRST       0.67      0.39      0.50       393
     NEUTRAL       0.53      0.22      0.31       418
          NO       0.60      0.92      0.73      1012
      SECOND       0.62      0.33      0.43       284

    accuracy                           0.60      2107
   macro avg       0.60      0.47      0.49      2107
weighted avg       0.60      0.60      0.56      2107



In [39]:
# 0: NO, 1: NEUTRAL, 2: FIRST, 3: SECOND

print(classification_report(y_true=df_texts.Score_num.tolist(), y_pred = df_texts.gpt_pred_num.tolist()))

              precision    recall  f1-score   support

           0       0.60      0.92      0.73      1012
           1       0.53      0.22      0.31       418
           2       0.67      0.39      0.50       393
           3       0.62      0.33      0.43       284

    accuracy                           0.60      2107
   macro avg       0.60      0.47      0.49      2107
weighted avg       0.60      0.60      0.56      2107

